# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import numpy as np
import pandas as pd
import cassandra
import re
import os
import glob
import json
import csv

from cassandra.cluster import Cluster

def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
print(filepath)

# Create a for loop to create a list of files and collect each filepath and 
# join the file path and roots with the subdirectories using glob
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace
/home/workspace/event_data


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 


csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName',
                     'length', 'level','location','sessionId','song','userId'])

    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6],
                         row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
# To establish connection and begin executing queries, need a session

try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkifyks 
        WITH REPLICATION = 
        {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)

except Exception as e:
    print(e)

#### Set Keyspace
    # TO-DO: Set KEYSPACE to the keyspace specified above

session.set_keyspace('sparkifyks')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [7]:
#Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338 and itemInSession = 4

In [8]:
# we need to create a table with only needed 5 columns from the csv file. Data types for columns easily understandable from the create table statement in code.
### Here the Primary Key has two fields: sessionId is the partition key, and itemInSession are clustering keys. 
## Partitioning is done by sessionId and within that partition, rows are ordered by the itemInSession.
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS songinfo_by_session_by_item (
            sessionId INT,
            itemInSession INT,
            artist TEXT,
            song TEXT,
            length FLOAT,
            PRIMARY KEY (sessionId, itemInSession));
    """)
except Exception as e:
    print(e)

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO songinfo_by_session_by_item (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
session.row_factory = pandas_factory

result = session.execute("""
    SELECT artist, song, length
    FROM songinfo_by_session_by_item
    WHERE
        sessionId=338 AND
        itemInSession=4;
""")

df = result._current_rows 
df

In [ ]:
#At sessionId=338 and itemInsession=4, the artist was 'Faithless', and the song was 'Music Matters (Mark Knight Dub)' with a length of 495.307312 seconds.

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [ ]:
## We need create a table with 7 columns from the csv file to be able to get this information.
## The Primary Key has three fields: userid and sessionid are the partition key, and itemInSession are clustering keys. 
## Partitioning is done by userid and sessionid, and within that partition, rows are ordered by the itemInSession.
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS songinfo_by_user_by_session (
            userId INT,
            sessionId INT,
            itemInSession INT,
            artist TEXT,
            song TEXT,
            firstName TEXT,
            lastName TEXT,
            PRIMARY KEY ((userId, sessionId), itemInSession));
    """)
except Exception as e:
    print(e)

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO songinfo_by_user_by_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [ ]:
session.row_factory = pandas_factory

result = session.execute("""
    SELECT artist, song, firstName, lastName
    FROM songinfo_by_user_by_session
    WHERE
        userId=10 AND
        sessionId=182
    ORDER BY itemInSession;
""")

df = result._current_rows 
df

In [ ]:
#In sessionId=182, userId=10 Sylvie Cruz listened to 4 songs: 'Keep on Keepin' On' by 'Down To The Bone', 'Greece 2000' by 'Three Drives', 'Kilometer' by 'Sebastien Tellier', and 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)' by 'Lonnie Gordon'.

In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## The Primary Key has two fields: song is the partition key, and userid are clustering keys. 
## Partitioning is done by song and within that partition, rows are ordered by the userid
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS userinfo_by_song (
            song TEXT,
            firstName TEXT,
            lastName TEXT,
            userId INT,
            PRIMARY KEY (song, firstName, lastName, userId));
    """)
except Exception as e:
    print(e)       

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO userinfo_by_song (song, firstName, lastName, userId)"
        query = query + " VALUES (%s, %s, %s, %s);"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))


In [ ]:
session.row_factory = pandas_factory

result = session.execute("""
    SELECT firstName, lastName
    FROM userinfo_by_song
    WHERE song='All Hands Against His Own';
""")

df = result._current_rows 
df

### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

In [ ]:
try:
    session.execute("DROP TABLE songinfo_by_session_by_item")
    session.execute("DROP TABLE songinfo_by_user_by_session")
    session.execute("DROP TABLE userinfo_by_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()